# Insurance ELT Driver

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/insurance_elt/lakelogic_driver.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/insurance_elt/lakelogic_driver.ipynb)

## Business Scenario

Multi-entity insurance data needs coordinated registry-driven execution across reference, bronze, silver, and gold layers.

## Value Proposition

- Registry-driven orchestration
- Layered execution with observability
- Reusable driver configuration

---

## Goals

1. Configure registries
2. Run the driver
3. Review summaries and metrics


In [1]:
# Run the driver (edit layers/window as needed)
import shutil
import subprocess
import sys
from pathlib import Path

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        candidate = base / name
        if candidate.exists():
            return candidate
        candidate = base / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
        candidate = base / "lakelogic" / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
    return cwd / name

BASE = resolve_example_dir("insurance_elt")
registry = BASE / "contracts" / "insurance" / "_registry.yaml"
reference_registry = BASE / "contracts" / "shared" / "reference" / "_registry.yaml"
gold_registry = BASE / "contracts" / "insurance" / "warehouse" / "_registry.yaml"

output_root = BASE / "output"
if output_root.exists():
    shutil.rmtree(output_root)

OUTPUT_DIR = output_root / "run_logs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def build_driver_cmd(args):
    exe = shutil.which("lakelogic-driver")
    if exe:
        return [exe] + args
    return [sys.executable, "-m", "lakelogic.cli.driver"] + args

cmd = build_driver_cmd([
    "--registry", str(registry),
    "--reference-registry", str(reference_registry),
    "--gold-registry", str(gold_registry),
    "--layers", "reference,bronze,silver,gold",
    "--max-workers", "1",
    "--window", "last_success",
    "--strict-layer-order",
    "--summary-table", "lakelogic.pipeline_runs",
    "--summary-backend", "sqlite",
    "--summary-database", str(OUTPUT_DIR / "lakelogic_pipeline_runs.db"),
    "--metrics-path", str(OUTPUT_DIR / "pipeline_metrics.json"),
])
print(" ".join(cmd))
subprocess.run(cmd, check=True)


C:\Program Files\Python313\python.exe -m lakelogic.cli.driver --registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\_registry.yaml --reference-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\shared\reference\_registry.yaml --gold-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\warehouse\_registry.yaml --layers reference,bronze,silver,gold --max-workers 1 --window last_success --strict-layer-order --summary-table lakelogic.pipeline_runs --summary-backend sqlite --summary-database D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\output\run_logs\lakelogic_pipeline_runs.db --metrics-path D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\output\run_logs\pipeline_metrics.json


CompletedProcess(args=['C:\\Program Files\\Python313\\python.exe', '-m', 'lakelogic.cli.driver', '--registry', 'D:\\Github\\_SaaS\\lakelogic\\examples\\06_advanced_workflows\\insurance_elt\\contracts\\insurance\\_registry.yaml', '--reference-registry', 'D:\\Github\\_SaaS\\lakelogic\\examples\\06_advanced_workflows\\insurance_elt\\contracts\\shared\\reference\\_registry.yaml', '--gold-registry', 'D:\\Github\\_SaaS\\lakelogic\\examples\\06_advanced_workflows\\insurance_elt\\contracts\\insurance\\warehouse\\_registry.yaml', '--layers', 'reference,bronze,silver,gold', '--max-workers', '1', '--window', 'last_success', '--strict-layer-order', '--summary-table', 'lakelogic.pipeline_runs', '--summary-backend', 'sqlite', '--summary-database', 'D:\\Github\\_SaaS\\lakelogic\\examples\\06_advanced_workflows\\insurance_elt\\output\\run_logs\\lakelogic_pipeline_runs.db', '--metrics-path', 'D:\\Github\\_SaaS\\lakelogic\\examples\\06_advanced_workflows\\insurance_elt\\output\\run_logs\\pipeline_metric

In [2]:
# Expose Prometheus metrics (run in a dedicated terminal for long-running scrape)
import subprocess

cmd = build_driver_cmd([
    "--registry", str(registry),
    "--reference-registry", str(reference_registry),
    "--gold-registry", str(gold_registry),
    "--layers", "reference,bronze,silver,gold",
    "--metrics-backend", "prometheus",
    "--metrics-host", "0.0.0.0",
    "--metrics-port", "9100",
])
print(" ".join(cmd))
# subprocess.run(cmd, check=True)


C:\Program Files\Python313\python.exe -m lakelogic.cli.driver --registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\_registry.yaml --reference-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\shared\reference\_registry.yaml --gold-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\warehouse\_registry.yaml --layers reference,bronze,silver,gold --metrics-backend prometheus --metrics-host 0.0.0.0 --metrics-port 9100


## Run a Single Entity
Use `--entities` to run only one or more entities without editing registries.


In [3]:
cmd = build_driver_cmd([
    "--registry", str(registry),
    "--reference-registry", str(reference_registry),
    "--gold-registry", str(gold_registry),
    "--layers", "bronze,silver,gold",
    "--entities", "policies",
])
print(" ".join(cmd))
# subprocess.run(cmd, check=True)


C:\Program Files\Python313\python.exe -m lakelogic.cli.driver --registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\_registry.yaml --reference-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\shared\reference\_registry.yaml --gold-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\warehouse\_registry.yaml --layers bronze,silver,gold --entities policies


## Reprocess Example
Use explicit date ranges for late-arriving data.


In [4]:
cmd = build_driver_cmd([
    "--registry", str(registry),
    "--reference-registry", str(reference_registry),
    "--gold-registry", str(gold_registry),
    "--layers", "silver,gold",
    "--reprocess-start-date", "2026-02-01",
    "--reprocess-end-date", "2026-02-05",
])
print(" ".join(cmd))
# subprocess.run(cmd, check=True)


C:\Program Files\Python313\python.exe -m lakelogic.cli.driver --registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\_registry.yaml --reference-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\shared\reference\_registry.yaml --gold-registry D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\insurance_elt\contracts\insurance\warehouse\_registry.yaml --layers silver,gold --reprocess-start-date 2026-02-01 --reprocess-end-date 2026-02-05
